In [7]:
import requests
import json
import pandas as pd

def get_building_data(address):
    client_id = "xn1cn78vuv"
    client_secret = "IjWjy6PKVbxNkHUsszGWOkPEuI15aditdrWrU9G9"

    geocode_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    search_url = "https://map.naver.com/v5/api/search"
    
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }

    # 위경도 수집
    geocode_params = {
        "query": address,
    }

    response = requests.get(geocode_url, headers=headers, params=geocode_params)
    try:
        response_data = json.loads(response.text)
    except json.JSONDecodeError:
        print(f"Invalid JSON response for url {geocode_url}")
        print("Response status code:", response.status_code)
        print("Response text:", response.text)
        return []

    if 'addresses' in response_data and len(response_data['addresses']) > 0:
        latitude = response_data['addresses'][0]['y']
        longitude = response_data['addresses'][0]['x']

        # 건물 이름, 건물에 포함된 상가수 수집
        search_params = {
            "caller": "pcweb",
            "query": address,
            "type": "all",
            "searchCoord": f"{longitude},{latitude}",
            "lang": "ko"
        }

        response = requests.get(search_url, params=search_params)
        try:
            search_data = json.loads(response.text)
        except json.JSONDecodeError:
            print(f"Invalid JSON response for url {search_url}")
            print("Response status code:", response.status_code)
            print("Response text:", response.text)
            return []

        if 'result' in search_data and 'address' in search_data['result']:
            building_data_list = []
            if 'jibunsAddress' in search_data['result']['address']:
                address_data = search_data['result']['address']['jibunsAddress']
                if 'list' in address_data:
                    for item in address_data['list']:
                        if 'siteRepName' in item and 'buildName' in item and 'siteCount' in item:
                            name = item['buildName']
                            site_count = item['siteCount']
                            latitude = item['y']
                            longitude = item['x']
                            building_data_list.append((name, site_count, latitude, longitude))
            return building_data_list
        else:
            return []
    else:
        return []
    

In [10]:
df = pd.read_excel("C:/Python_practice/DL_project/gwangjin_contest/최종/광진구_클러스터링.xlsx", sheet_name='clu2_300')
df.rename(columns = {'df_2_지번주소(인구밀도_기준_정렬)' : '지번주소'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   지번주소                  300 non-null    object 
 1   도로명주소                 300 non-null    object 
 2   위도                    300 non-null    float64
 3   경도                    300 non-null    float64
 4   소지역코드                 300 non-null    int64  
 5   인구밀도                  300 non-null    float64
 6   총 사업체수                300 non-null    int64  
 7   반경 1km 이내 불법주정차 단속 수  300 non-null    int64  
 8   주차장과의최단거리             300 non-null    int64  
 9   토지면적                  300 non-null    float64
 10  공시지가_기준년월             300 non-null    object 
 11  공시지가(원/면적)            300 non-null    int64  
 12  지하철이름                 300 non-null    object 
 13  역과의최단거리               300 non-null    int64  
 14  반경 1km 이내 주차장수        300 non-null    int64  
 15  Cluster               3

In [25]:
# api 반환 내용 확인
test = get_building_data('서울특별시 광진구 자양동 783-1')
print(test)

{'result': {'type': 'address', 'metaInfo': {'pageId': '629bbdc7-070a-4608-87ad-732ad5037f88', 'sessionId': None, 'rcode': None, 'searchedQuery': '서울특별시 광진구 자양동 783-1', 'correction': None, 'displayCorrectAnswer': True}, 'address': {'subType': 'jibun-address', 'roadAddress': None, 'jibunsAddress': {'page': 1, 'totalCount': 1, 'isExtendedSearch': False, 'list': [{'index': '0', 'rank': '1', 'id': '09215105', 'name': '서울특별시 광진구 자양동 783-1(아차산로36길 39)', 'fullAddress': '서울특별시 광진구 자양동 783-1', 'koreanAddress': '서울특별시 광진구 자양동 783-1', 'englishAddress': '39, Achasan-ro 36-gil, Gwangjin-gu, Seoul, Republic of Korea', 'addressElements': {'sido': '서울특별시', 'sigugun': '광진구', 'dongmyun': '자양동', 'ri': '', 'roadName': '아차산로36길', 'buildNumber': '39', 'jibun': '783-1', 'buildName': '자양7차우성아파트', 'roadGroupId': None, 'zipcode': '05067', 'bcode': '1121510500', 'hcode': ''}, 'shortAddress': ['서울 광진구', '자양동 783-1'], 'siteSearchQuery': '서울특별시 광진구 자양동 783-1', 'display': '서울특별시 광진구 <em><strong>자양동 783-1</strong></em

In [28]:
# 함수 test
test2 = get_building_data('서울특별시 광진구 자양동 783-1')
print(test2)

[('자양7차우성아파트', '37', '37.5363569', '127.0746096')]


In [38]:
# 스크립트가 빈 문자열이나 유효하지 않은 JSON 형식의 응답을 받을 때 test
test3 = get_building_data('서울특별시 광진구 자양동 595-10')
print(test3)

Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
[]


In [11]:
# 건물 데이터를 가져올 DataFrame을 생성합니다.
building_data_df = pd.DataFrame(columns=['지번', '건물name', '건물_내_상가수', 'latitude(y)', 'longitude(x)'])

# 각 건물에 대해 데이터를 가져옵니다.
for index, row in df.iterrows():
    building_data_list = get_building_data(row['지번주소'])

    # 건물 이름, 유형 및 위도와 경도 데이터를 DataFrame에 추가합니다.
    for building_data in building_data_list:
        new_row = {
            '지번': row['지번주소'],
            '건물name': building_data[0],
            '건물_내_상가수': building_data[1],
            'latitude(y)': building_data[2],
            'longitude(x)': building_data[3]
        }
        building_data_df = building_data_df.append(new_row, ignore_index=True)

# 결과를 출력합니다.
print(building_data_df)

Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5

KeyboardInterrupt: 

In [4]:
building_data_df

,지번,건물name,건물_내_상가수,latitude(y),longitude(x)


In [34]:
building_data_df.to_excel(path+'clustering_popup창_v1.xlsx', index=False)

<hr>

## 모서리 좌표 수집
대한민국 최신 행정구역(SHP) -> 동까지만 구할 수 있어 중단 

In [4]:
# pip install geopandas

In [9]:
# # 대한민국 최신 행정구역(SHP) 다운로드  (http://www.gisdeveloper.co.kr/?p=2332)
# 대한민국의 행정구역에 대한 시도, 시군구, 읍면동, 리에 대한 공간 데이터
# 2023년02월 동 데이터 (한 파일에 '.shp, .shx, .dbf' 파일이 함께 존재해야 함)

import geopandas as gpd

# Shapefile 데이터를 읽어오기
path = 'C:/Python_practice/DL_project/gwangjin_contest/crawling/clustering_final_buildingtype_collect/'
gdf = gpd.read_file(path + "emd.shp", encoding='EUC-KR')

# 열 이름 확인
print(gdf.columns)

# 지역명 확인
print(gdf['EMD_KOR_NM'].unique())

Index(['EMD_CD', 'EMD_ENG_NM', 'EMD_KOR_NM', 'geometry'], dtype='object')
['청운동' '신교동' '궁정동' ... '남원읍' '성산읍' '표선면']


In [10]:
# 필요한 지역을 선택합니다. 예를 들어, 지역명으로 선택할 수 있습니다.
selected_area = gdf[gdf['EMD_KOR_NM'] == '자양동'].geometry.iloc[0]

# 모서리 좌표를 추출합니다.
corner_coordinates = []
for point in selected_area.exterior.coords:
    corner_coordinates.append((point[1], point[0]))

print(corner_coordinates)

[(1948937.413097647, 962325.5581222958), (1948919.1852532038, 962383.0197806115), (1948937.156363192, 962384.1716478193), (1948933.9759526146, 962394.8467541233), (1948933.4073920988, 962396.6900435161), (1948912.964262474, 962461.5291656372), (1948929.875157197, 962463.5510818989), (1948959.9970714082, 962466.5147022159), (1948962.1587576969, 962467.3628457566), (1948973.2624861863, 962471.7150874273), (1948980.0258489673, 962474.3709519642), (1948977.0729422034, 962486.2548073608), (1948968.9582219834, 962492.059583351), (1948964.3655381147, 962495.3439883032), (1948968.502336509, 962496.4075247498), (1948966.1011570208, 962512.6406641869), (1948965.6792696796, 962520.0036477719), (1948964.9303994665, 962533.0437544063), (1948964.4446220414, 962541.5219780455), (1948963.7825940596, 962553.0771054626), (1948963.1349703434, 962564.3664094717), (1948962.6029231413, 962573.6530822582), (1948966.3306517554, 962572.87314519), (1948965.5350964614, 962577.560156906), (1948960.3387632826, 962

<hr>

## 토지이음에서 필지고유번호로 모서리 좌표 수집하기 (성공)

[1] 네이버 api로 위경도 수집

In [12]:
df = pd.read_excel("C:/Python_practice/DL_project/gwangjin_contest/최종/광진구_클러스터링.xlsx", sheet_name='clu2_300')
df.rename(columns = {'df_2_지번주소(인구밀도_기준_정렬)' : '지번주소'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   지번주소                  300 non-null    object 
 1   도로명주소                 300 non-null    object 
 2   위도                    300 non-null    float64
 3   경도                    300 non-null    float64
 4   소지역코드                 300 non-null    int64  
 5   인구밀도                  300 non-null    float64
 6   총 사업체수                300 non-null    int64  
 7   반경 1km 이내 불법주정차 단속 수  300 non-null    int64  
 8   주차장과의최단거리             300 non-null    int64  
 9   토지면적                  300 non-null    float64
 10  공시지가_기준년월             300 non-null    object 
 11  공시지가(원/면적)            300 non-null    int64  
 12  지하철이름                 300 non-null    object 
 13  역과의최단거리               300 non-null    int64  
 14  반경 1km 이내 주차장수        300 non-null    int64  
 15  Cluster               3

In [15]:
import numpy as np
new_df = pd.read_excel("C:/Python_practice/DL_project/gwangjin_contest/최종/광진구_클러스터링.xlsx", sheet_name='clu2_300')[['지번주소']]
new_df['위도(lat)'] = np.nan
new_df['경도(long)'] = np.nan
new_df.head(3)

,지번주소,위도(lat),경도(long)
0,서울특별시 광진구 자양동 673,NaN,NaN
1,서울특별시 광진구 자양동 672,NaN,NaN
2,서울특별시 광진구 자양동 645-5,NaN,NaN


In [16]:
# 네이버 API 활용하여 위경도 따오기
import requests
import json
import pandas as pd

def get_latitude_longitude(address):
    client_id = "xn1cn78vuv"
    client_secret = "IjWjy6PKVbxNkHUsszGWOkPEuI15aditdrWrU9G9"

    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }

    params = {
        "query": address,
    }

    response = requests.get(url, headers=headers, params=params)
    response_data = json.loads(response.text)

    if 'addresses' in response_data and len(response_data['addresses']) > 0:
        latitude = response_data['addresses'][0]['y']
        longitude = response_data['addresses'][0]['x']
        return latitude, longitude
    else:
        return None, None

for index, row in new_df.iterrows():
    latitude, longitude = get_latitude_longitude(row['지번주소'])
    new_df.at[index, '위도(lat)'] = latitude
    new_df.at[index, '경도(long)'] = longitude

In [20]:
path = 'C:/Python_practice/DL_project/gwangjin_contest/crawling/clustering_final_buildingtype_collect/'
new_df.to_excel(path+"clustering2_300_위경도.xlsx", index=False)

[2] 필지고유번호 계산

In [73]:
code_df = pd.read_excel("C:/Python_practice/DL_project/gwangjin_contest/최종/광진구_클러스터링.xlsx", sheet_name='필지고유번호')
code_df['필지고유번호'] = np.nan
code_df.head(3)

,지번주소,동,본번,부번,필지고유번호
0,서울특별시 광진구 자양동 673,자양동,673,0,NaN
1,서울특별시 광진구 자양동 672,자양동,672,0,NaN
2,서울특별시 광진구 자양동 645-5,자양동,645,5,NaN


In [74]:
for index, row in code_df.iterrows():
    code = ''
    dong = row['동']
    bon = row['본번']
    bubun = row['부번']

    # 동에 따른 행정표준코드
    dong_code = ''
    if dong == "자양동" : 
        dong_code = "1121510500"
    elif dong == "중곡동" : 
        dong_code = "1121510100"
    
    # 부번이 0인 경우 0000으로 변경
    if bubun == 0:
        bubun = '0000'
    # 부번이 4자리 숫자가 아닌 경우 4자리 숫자로 변경
    else:
        bubun = '{0:04d}'.format(bubun)

    code = f"{dong_code}10{bon}{bubun}"
    code_df['필지고유번호'][index] = code
code_df

C:\Users\dohee\AppData\Local\Temp\ipykernel_2412\3749664316.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  code_df['필지고유번호'][index] = code


,지번주소,동,본번,부번,필지고유번호
0,서울특별시 광진구 자양동 673,자양동,673,0,1121510500106730000
1,서울특별시 광진구 자양동 672,자양동,672,0,1121510500106720000
2,서울특별시 광진구 자양동 645-5,자양동,645,5,1121510500106450005
3,서울특별시 광진구 자양동 667-6,자양동,667,6,1121510500106670006
4,서울특별시 광진구 자양동 667-8,자양동,667,8,1121510500106670008
...,...,...,...,...,...
295,서울특별시 광진구 자양동 663-11,자양동,663,11,1121510500106630011
296,서울특별시 광진구 자양동 663-12,자양동,663,12,1121510500106630012
297,서울특별시 광진구 자양동 663-13,자양동,663,13,1121510500106630013
298,서울특별시 광진구 자양동 663-20,자양동,663,20,1121510500106630020


In [75]:
code_df.to_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/cluster2_300_필지고유번호.xlsx', index=False)

In [ ]:
# 참고 : https://devlog.jwgo.kr/2020/01/20/how-to-get-pnu/

[3] 필지고유번호로 토지이음 사이트에서 api를 통한 모서리 좌표 수집

In [76]:
code_df['모서리_좌표'] = np.nan
code_df.columns

Index(['지번주소', '동', '본번', '부번', '필지고유번호', '모서리_좌표'], dtype='object')

In [77]:
code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지번주소    300 non-null    object 
 1   동       300 non-null    object 
 2   본번      300 non-null    int64  
 3   부번      300 non-null    int64  
 4   필지고유번호  300 non-null    object 
 5   모서리_좌표  0 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 14.2+ KB


In [78]:
import json
import math
import time

# 모서리_좌표 열의 데이터 유형을 'object'로 변경
code_df['모서리_좌표'] = code_df['모서리_좌표'].astype('object')

url = 'http://119.196.18.3:8004/MapPlan/MapPlan?req=search&version=20230109&layer=FA&code='

for index, row in code_df.iterrows():
    add_code = row['필지고유번호']
    response = requests.get(url + add_code)

    # coordinates를 저장할 빈 리스트 생성
    all_coordinates = []

    if response.status_code == 200:
        data = json.loads(response.text)
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates'][0]
            all_coordinates.append(coordinates)
            code_df.at[index, '모서리_좌표'] = all_coordinates
        else:
            code_df.at[index, '모서리_좌표'] = 0
    else:
        code_df.at[index, '모서리_좌표'] = 0  


[4] 모서리 좌표 folium 라이브러리의 좌표계로 변환하기

In [45]:
# pip install pyproj

In [134]:
code_df = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/cluster2_300_모서리좌표.xlsx', index_col=False)
code_df['좌표계_변환'] = np.nan
code_df['좌표계_변환'] = code_df['좌표계_변환'].astype('object')

In [150]:
code_df.head(3)

,지번주소,동,본번,부번,필지고유번호,모서리_좌표,좌표계_변환
0,서울특별시 광진구 자양동 673,자양동,673,0,1121510500106730000,"[[[963470.22922, 1947798.31742], [963464.71644...",NaN
1,서울특별시 광진구 자양동 672,자양동,672,0,1121510500106720000,"[[[963327.6845, 1947768.39449], [963331.29215,...",NaN
2,서울특별시 광진구 자양동 645-5,자양동,645,5,1121510500106450005,"[[[963135.04342, 1947773.31397], [963136.55665...",NaN


In [153]:
# 좌표계 변환
# # 대한민국의 좌표계 중 하나인 UTM-K (a.k.a. EPSG:5179) 좌표계 -> Folium 라이브러리 WGS84 좌표계 (EPSG:4326)

from pyproj import Transformer

def coord_to_latlng(coord):
    """
    좌표 변환 함수: UTM-K (EPSG:5179) -> WGS84 (EPSG:4326)
    """
    transformer = Transformer.from_crs('epsg:5179', 'epsg:4326', always_xy=True)
    lng, lat = transformer.transform(coord[0], coord[1])
    return [lat, lng]

# 좌표 변환
series = code_df['모서리_좌표']
for index in range(len(series)) :
    for coordinates in series[index] : 
        coordinates_wgs84 = [coord_to_latlng(coord) for coord in coordinates]
        code_df.at[index, '좌표계_변환'] = coordinates_wgs84    

In [154]:
code_df.head(3)

,지번주소,동,본번,부번,필지고유번호,모서리_좌표,좌표계_변환
0,서울특별시 광진구 자양동 673,자양동,673,0,1121510500106730000,"[[[963470.22922, 1947798.31742], [963464.71644...","[[37.528768937685, 127.08656315765265], [37.52..."
1,서울특별시 광진구 자양동 672,자양동,672,0,1121510500106720000,"[[[963327.6845, 1947768.39449], [963331.29215,...","[[37.52849357214731, 127.08495139733036], [37...."
2,서울특별시 광진구 자양동 645-5,자양동,645,5,1121510500106450005,"[[[963135.04342, 1947773.31397], [963136.55665...","[[37.52853023105759, 127.08277093692556], [37...."


[5] Folium 지도에 표시

In [155]:
merged_df = pd.merge(code_df, new_df, on='지번주소', how='left')
merged_df

,지번주소,동,본번,부번,필지고유번호,모서리_좌표,좌표계_변환,위도(lat),경도(long)
0,서울특별시 광진구 자양동 673,자양동,673,0,1121510500106730000,"[[[963470.22922, 1947798.31742], [963464.71644...","[[37.528768937685, 127.08656315765265], [37.52...",37.5284419,127.0859202
1,서울특별시 광진구 자양동 672,자양동,672,0,1121510500106720000,"[[[963327.6845, 1947768.39449], [963331.29215,...","[[37.52849357214731, 127.08495139733036], [37....",37.5281456,127.0846890
2,서울특별시 광진구 자양동 645-5,자양동,645,5,1121510500106450005,"[[[963135.04342, 1947773.31397], [963136.55665...","[[37.52853023105759, 127.08277093692556], [37....",37.5281212,127.0824168
3,서울특별시 광진구 자양동 667-6,자양동,667,6,1121510500106670006,"[[[963305.65953, 1947894.1437], [963313.27944,...","[[37.5296261218857, 127.08469584585208], [37.5...",37.5295417,127.0846818
4,서울특별시 광진구 자양동 667-8,자양동,667,8,1121510500106670008,"[[[963294.85468, 1947884.91447], [963301.23591...","[[37.52954250520042, 127.08457402156752], [37....",37.5294695,127.0845523
...,...,...,...,...,...,...,...,...,...
295,서울특별시 광진구 자양동 663-11,자양동,663,11,1121510500106630011,"[[[963184.26924, 1947968.41921], [963176.41789...","[[37.53029075614659, 127.08331826746432], [37....",37.5303358,127.0832601
296,서울특별시 광진구 자양동 663-12,자양동,663,12,1121510500106630012,"[[[963193.03705, 1947991.26409], [963199.31009...","[[37.53049701583824, 127.08341635423629], [37....",37.5304164,127.0834062
297,서울특별시 광진구 자양동 663-13,자양동,663,13,1121510500106630013,"[[[963201.24433, 1948006.42484], [963202.20217...","[[37.53063399307986, 127.08350848268658], [37....",37.5305097,127.0835058
298,서울특별시 광진구 자양동 663-20,자양동,663,20,1121510500106630020,"[[[963214.14941, 1948006.41642], [963215.39849...","[[37.53063443219414, 127.08365454039398], [37....",37.5306255,127.0835993


In [162]:
merged_df.info()
merged_df.to_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/cluster2_300_full좌표.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지번주소    300 non-null    object 
 1   동       300 non-null    object 
 2   본번      300 non-null    int64  
 3   부번      300 non-null    int64  
 4   필지고유번호  300 non-null    int64  
 5   모서리_좌표  300 non-null    object 
 6   좌표계_변환  300 non-null    object 
 7   위도      300 non-null    float64
 8   경도      300 non-null    float64
dtypes: float64(2), int64(3), object(4)
memory usage: 23.4+ KB


In [172]:
load_width_df = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/광진구_클러스터링_도로폭추가.xlsx', index_col=False)
load_width_df.drop(['위도', '경도', '소지역코드', '공시지가_기준년월', '공시지가(원/면적)', '역과의최단거리', '반경 1km 이내 주차장수', 'Cluster'], 
                    axis=1, inplace = True)
load_width_df.head(5)

,지번주소,도로명주소,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,지하철이름,도로폭
0,서울특별시 광진구 광장동 185-6,서울특별시 광진구 아차산로78길 175 (광장동),12871.2,191,7,73,300.0,광나루역,13.0
1,서울특별시 광진구 광장동 231-1,서울특별시 광진구 아차산로 567 (광장동),17062.9,36,10,173,646.0,광나루역,37.0
2,서울특별시 광진구 광장동 258-10,서울특별시 광진구 광장로1길 22-7 (광장동),13372.8,49,8,30,93.0,광나루역,7.0
3,서울특별시 광진구 광장동 258-11,서울특별시 광진구 광장로1길 22-5 (광장동),13372.8,49,8,25,99.0,광나루역,7.0
4,서울특별시 광진구 광장동 258-12,서울특별시 광진구 광장로1길 22-3 (광장동),13372.8,49,8,24,99.0,광나루역,7.0


In [173]:
merged_df_2 = pd.merge(merged_df, load_width_df, on='지번주소', how='left')
merged_df_2.drop(['동', '본번', '부번', '필지고유번호', '모서리_좌표', '좌표계_변환'], 
                    axis=1, inplace = True)
merged_df_2.head(5)

,지번주소,위도,경도,도로명주소,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,지하철이름,도로폭
0,서울특별시 광진구 자양동 673,37.528442,127.085920,"서울특별시 광진구 자양로3가길 43 (자양동, 현대강변아파트)",69636.0,10,16,99,7711.1,구의역,7.0
1,서울특별시 광진구 자양동 672,37.528146,127.084689,"서울특별시 광진구 뚝섬로56가길 28 (자양동, 한솔리베르아파트)",89457.4,11,16,127,5028.4,뚝섬유원지역,7.0
2,서울특별시 광진구 자양동 645-5,37.528121,127.082417,"서울특별시 광진구 뚝섬로56길 72 (자양동, 자양동현대아파트)",72831.9,7,10,41,6384.8,뚝섬유원지역,9.0
3,서울특별시 광진구 자양동 667-6,37.529542,127.084682,서울특별시 광진구 뚝섬로58길 66 (자양동),44316.5,38,18,66,207.6,구의역,7.0
4,서울특별시 광진구 자양동 667-8,37.529469,127.084552,서울특별시 광진구 뚝섬로58길 68 (자양동),44316.5,38,18,72,172.5,구의역,7.0


In [157]:
merged_df.rename(columns = {'위도(lat)' : '위도'}, inplace = True)
merged_df.rename(columns = {'경도(long)' : '경도'}, inplace = True)
parking_lot = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/광진구_공영주차장_230424.xlsx')

In [161]:
import folium

def plot_data_on_map(data, parking_lot, zoom_start=14):
    """
    주어진 데이터를 지도에 시각화하는 함수입니다.
    data: 지도에 표시할 데이터프레임
    parking_lot: 주차장 데이터프레임
    zoom_start: 지도의 초기 확대 수준 (기본값: 14)
    """

    # 위도와 경도의 데이터 유형을 float로 변환
    data['위도'] = data['위도'].astype(float)
    data['경도'] = data['경도'].astype(float)

    # 위도와 경도의 평균값을 사용하여 지도의 초기 위치 설정
    map_center_lat = data['위도'].mean()
    map_center_lon = data['경도'].mean()

    # 지도 생성
    m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=zoom_start)

    # 지도에 마커 추가
    for index, row in data.iterrows():
        lat, lon = row['위도'], row['경도']
        jibun_address = row['지번주소']  
        popup_text = f"지번 주소: {jibun_address}"
        folium.Marker(location=[lat, lon], popup=folium.Popup(popup_text, max_width=200)).add_to(m)

        # 면적 출력 (변환된 좌표로 다각형 생성)
        # for content in row['좌표계_변환']: 
        folium.Polygon(locations=row['좌표계_변환'], color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(m)  # 지도에 다각형 추가

    # 지도에 주차장의 빨간색 원 추가
    for index, row in parking_lot.iterrows():
        lat, lon = row['위도'], row['경도']
        folium.CircleMarker(location=[lat, lon], radius=5, color='red', fill=True, fill_color='red').add_to(m)

    # 지도 출력
    return m

# 함수 사용 
map_with_markers = plot_data_on_map(merged_df, parking_lot)
map_with_markers

In [163]:
map_with_markers.save('C:/Python_practice/DL_project/gwangjin_contest/최종/cluster2_top300_map.html')

<hr>

# 지도 upgrade

In [175]:
merged_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   지번주소                  300 non-null    object 
 1   위도                    300 non-null    float64
 2   경도                    300 non-null    float64
 3   도로명주소                 300 non-null    object 
 4   인구밀도                  300 non-null    float64
 5   총 사업체수                300 non-null    int64  
 6   반경 1km 이내 불법주정차 단속 수  300 non-null    int64  
 7   주차장과의최단거리             300 non-null    int64  
 8   토지면적                  300 non-null    float64
 9   지하철이름                 300 non-null    object 
 10  도로폭                   300 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 28.1+ KB


In [177]:
merged_df_2['건물이름'] = np.nan
merged_df_2.head(3)

,지번주소,위도,경도,도로명주소,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,지하철이름,도로폭,건물이름
0,서울특별시 광진구 자양동 673,37.528442,127.085920,"서울특별시 광진구 자양로3가길 43 (자양동, 현대강변아파트)",69636.0,10,16,99,7711.1,구의역,7.0,NaN
1,서울특별시 광진구 자양동 672,37.528146,127.084689,"서울특별시 광진구 뚝섬로56가길 28 (자양동, 한솔리베르아파트)",89457.4,11,16,127,5028.4,뚝섬유원지역,7.0,NaN
2,서울특별시 광진구 자양동 645-5,37.528121,127.082417,"서울특별시 광진구 뚝섬로56길 72 (자양동, 자양동현대아파트)",72831.9,7,10,41,6384.8,뚝섬유원지역,9.0,NaN


In [181]:
import requests
import json
import pandas as pd

def get_building_data_with_lat_lng(address, latitude, longitude):
    """
    naver api를 통해 건물 이름과 건물 내 상가수를 가져오는 함수
    """

    search_url = "https://map.naver.com/v5/api/search"

    # 건물 이름, 건물에 포함된 상가수 수집
    search_params = {
        "caller": "pcweb",
        "query": address,
        "type": "all",
        "searchCoord": f"{longitude},{latitude}",
        "lang": "ko"
    }

    response = requests.get(search_url, params=search_params)
    try:
        search_data = json.loads(response.text)
    except json.JSONDecodeError:
        print(f"Invalid JSON response for url {search_url}")
        print("Response status code:", response.status_code)
        print("Response text:", response.text)
        return []

    if 'result' in search_data and 'address' in search_data['result']:
        building_data_list = []
        if 'jibunsAddress' in search_data['result']['address']:
            address_data = search_data['result']['address']['jibunsAddress']
            if 'list' in address_data:
                for item in address_data['list']:
                    if 'siteRepName' in item and 'buildName' in item and 'siteCount' in item:
                        name = item['buildName']
                        site_count = item['siteCount']
                        building_data_list.append((name, site_count))
        return building_data_list
    else:
        return []

In [182]:
# 건물 데이터를 가져올 DataFrame을 생성합니다.
building_data_df = pd.DataFrame(columns=['지번', '건물name', '건물_내_상가수'])

# 각 건물에 대해 데이터를 가져옵니다.
for index, row in merged_df_2.iterrows():
    building_data_list = get_building_data_with_lat_lng(row['지번주소'], row['위도'], row['경도'])

    # 건물 이름, 유형 및 위도와 경도 데이터를 DataFrame에 추가합니다.
    for building_data in building_data_list:
        new_row = {
            '지번': row['지번주소'],
            '건물name': building_data[0],
            '건물_내_상가수': building_data[1]
        }
        building_data_df = building_data_df.append(new_row, ignore_index=True)

# 결과를 출력합니다.
print(building_data_df)

C:\Users\dohee\AppData\Local\Temp\ipykernel_2412\523580050.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  building_data_df = building_data_df.append(new_row, ignore_index=True)
C:\Users\dohee\AppData\Local\Temp\ipykernel_2412\523580050.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  building_data_df = building_data_df.append(new_row, ignore_index=True)
C:\Users\dohee\AppData\Local\Temp\ipykernel_2412\523580050.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  building_data_df = building_data_df.append(new_row, ignore_index=True)
C:\Users\dohee\AppData\Local\Temp\ipykernel_2412\523580050.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.c

Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5/
Invalid JSON response for url https://map.naver.com/v5/api/search
Response status code: 503
Response text: Moved Permanently. Redirecting to https://map.naver.com/v5

KeyboardInterrupt: 

In [196]:
building_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   지번        99 non-null     object
 1   건물이름      99 non-null     object
 2   건물_내_상가수  99 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [187]:
building_data_df.rename(columns = {'건물name' : '건물이름'}, inplace = True)
building_data_df['건물_내_상가수'] = building_data_df['건물_내_상가수'].astype(int)

In [193]:
for index, row in building_data_df.iterrows():
    if building_data_df.at[index, '건물이름'] == '':
        if building_data_df.at[index, '건물_내_상가수'] == 0:
            building_data_df.at[index, '건물이름'] = '주거 목적 건물'
        else:
            building_data_df.at[index, '건물이름'] = '상업 목적 건물'

In [195]:
building_data_df.to_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/건물이름.xlsx', index=False)

In [200]:
building_data = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/건물이름.xlsx', index_col=False)
building_data.rename(columns = {'지번' : '지번주소'}, inplace = True)
building_data.head(2)

,지번주소,건물이름,건물_내_상가수
0,서울특별시 광진구 자양동 673,현대강변아파트,3
1,서울특별시 광진구 자양동 672,한솔리베르아파트,6


In [201]:
merged_df_3 = pd.merge(merged_df_2, building_data, on='지번주소', how='left')
merged_df_3.head(5)

,지번주소,위도,경도,도로명주소,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,지하철이름,도로폭,건물이름_x,건물이름_y,건물_내_상가수
0,서울특별시 광진구 자양동 673,37.528442,127.085920,"서울특별시 광진구 자양로3가길 43 (자양동, 현대강변아파트)",69636.0,10,16,99,7711.1,구의역,7.0,NaN,현대강변아파트,3.0
1,서울특별시 광진구 자양동 672,37.528146,127.084689,"서울특별시 광진구 뚝섬로56가길 28 (자양동, 한솔리베르아파트)",89457.4,11,16,127,5028.4,뚝섬유원지역,7.0,NaN,한솔리베르아파트,6.0
2,서울특별시 광진구 자양동 645-5,37.528121,127.082417,"서울특별시 광진구 뚝섬로56길 72 (자양동, 자양동현대아파트)",72831.9,7,10,41,6384.8,뚝섬유원지역,9.0,NaN,자양동현대아파트,5.0
3,서울특별시 광진구 자양동 667-6,37.529542,127.084682,서울특별시 광진구 뚝섬로58길 66 (자양동),44316.5,38,18,66,207.6,구의역,7.0,NaN,자양중앙교회,1.0
4,서울특별시 광진구 자양동 667-8,37.529469,127.084552,서울특별시 광진구 뚝섬로58길 68 (자양동),44316.5,38,18,72,172.5,구의역,7.0,NaN,상업 목적 건물,4.0


In [204]:
del merged_df_3['건물이름_x']

In [207]:
merged_df_3.rename(columns = {'건물이름_y' : '건물이름'}, inplace = True)
merged_df_3.head(2)

,지번주소,위도,경도,도로명주소,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,지하철이름,도로폭,건물이름,건물_내_상가수
0,서울특별시 광진구 자양동 673,37.528442,127.085920,"서울특별시 광진구 자양로3가길 43 (자양동, 현대강변아파트)",69636.0,10,16,99,7711.1,구의역,7.0,현대강변아파트,3.0
1,서울특별시 광진구 자양동 672,37.528146,127.084689,"서울특별시 광진구 뚝섬로56가길 28 (자양동, 한솔리베르아파트)",89457.4,11,16,127,5028.4,뚝섬유원지역,7.0,한솔리베르아파트,6.0


In [208]:
merged_df_3.to_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/건물이름_map.xlsx', index=False)

In [212]:
final_map = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/최종/건물이름_map.xlsx', index_col=False)
final_map.head(3)

,지번주소,위도,경도,좌표계_변환,도로명주소,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,지하철이름,도로폭,건물이름,건물_내_상가수
0,서울특별시 광진구 자양동 673,37.528442,127.085920,"[[37.528768937685, 127.08656315765265], [37.52...","서울특별시 광진구 자양로3가길 43 (자양동, 현대강변아파트)",69636.0,10,16,99,7711.1,구의역,7,현대강변아파트,3
1,서울특별시 광진구 자양동 672,37.528146,127.084689,"[[37.52849357214731, 127.08495139733036], [37....","서울특별시 광진구 뚝섬로56가길 28 (자양동, 한솔리베르아파트)",89457.4,11,16,127,5028.4,뚝섬유원지역,7,한솔리베르아파트,6
2,서울특별시 광진구 자양동 645-5,37.528121,127.082417,"[[37.52853023105759, 127.08277093692556], [37....","서울특별시 광진구 뚝섬로56길 72 (자양동, 자양동현대아파트)",72831.9,7,10,41,6384.8,뚝섬유원지역,9,자양동현대아파트,5


In [214]:
final_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   지번주소                  300 non-null    object 
 1   위도                    300 non-null    float64
 2   경도                    300 non-null    float64
 3   좌표계_변환                300 non-null    object 
 4   도로명주소                 300 non-null    object 
 5   인구밀도                  300 non-null    float64
 6   총 사업체수                300 non-null    int64  
 7   반경 1km 이내 불법주정차 단속 수  300 non-null    int64  
 8   주차장과의최단거리             300 non-null    int64  
 9   토지면적                  300 non-null    float64
 10  지하철이름                 300 non-null    object 
 11  도로폭                   300 non-null    int64  
 12  건물이름                  300 non-null    object 
 13  건물_내_상가수              300 non-null    object 
dtypes: float64(4), int64(4), object(6)
memory usage: 32.9+ KB


In [216]:
import folium

def plot_data_on_map(data, parking_lot, zoom_start=14):
    """
    주어진 데이터를 지도에 시각화하는 함수입니다.
    data: 지도에 표시할 데이터프레임
    parking_lot: 주차장 데이터프레임
    zoom_start: 지도의 초기 확대 수준 (기본값: 14)
    """

    # 위도와 경도의 데이터 유형을 float로 변환
    data['위도'] = data['위도'].astype(float)
    data['경도'] = data['경도'].astype(float)

    # 위도와 경도의 평균값을 사용하여 지도의 초기 위치 설정
    map_center_lat = data['위도'].mean()
    map_center_lon = data['경도'].mean()

    # 지도 생성
    m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=zoom_start)

    # 지도에 마커 추가
    for index, row in data.iterrows():
        lat, lon = row['위도'], row['경도']
        jibun_address = row['지번주소']  
        road_address = row['도로명주소']
        building_name = row['건물이름']
        building_num = row['건물_내_상가수']
        road_width = row['도로폭']  

        # 팝업에 표시할 HTML 코드 생성
        popup_text = f"""
        <div>
        <p><b>지번 주소:</b> {jibun_address}</p>
        <p><b>도로명 주소:</b> {road_address}</p>
        <p><b>건물 이름:</b> {building_name}</p>
        <p><b>건물 내 상가 수:</b> {building_num} 개</p>
        <p><b>도로 폭:</b> {road_width} m²</p>
        </div>
        """
        # 팝업에 HTML 코드 적용
        folium.Marker(location=[lat, lon], popup=folium.Popup(popup_text, max_width=200)).add_to(m)
  
        # 문자열을 리스트로 변환
        coord_converted = ast.literal_eval(row['좌표계_변환'])
        # 면적 출력 (변환된 좌표로 다각형 생성)
        folium.Polygon(locations=coord_converted, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(m)
        # folium.Polygon(locations=row['좌표계_변환'], color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(m)  # 지도에 다각형 추가

    # 지도에 주차장의 빨간색 원 추가
    for index, row in parking_lot.iterrows():
        lat, lon = row['위도'], row['경도']
        folium.CircleMarker(location=[lat, lon], radius=5, color='red', fill=True, fill_color='red').add_to(m)

    # 지도 출력
    return m

# 함수 사용 
map_with_markers = plot_data_on_map(final_map, parking_lot)
map_with_markers

In [217]:
map_with_markers.save('C:/Python_practice/DL_project/gwangjin_contest/최종/cluster2_top300_map_final.html')